# Setup

In [ ]:
!pip install ultralytics boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.4/778.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86

In [ ]:
from ultralytics import YOLO

In [ ]:
from os import listdir
from os.path import isfile, join
import sys
import os
from PIL import Image
import shutil
import numpy as np
import yaml
import argparse
from concurrent import futures
import os
import re
import sys
import boto3
import botocore
import tqdm

In [ ]:
import albumentations as A
import random as rnd

# Data preparation

In [ ]:
classes = ["Ci", "Cs", "Cc", "Ac", "As", "Cu", "Cb", "Ns", "Sc", "St", "Ct"]

In [ ]:
!unzip /content/no_night_CCSN.zip -d /content/no_night_ccsn

Archive:  /content/no_night_CCSN.zip
warning [/content/no_night_CCSN.zip]:  7857195 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  7857195
  (attempting to re-compensate)
error: invalid zip file with overlapped components (possible zip bomb)


In [ ]:
!unzip CCSN.zip -d /content/ccsn

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
  inflating: /content/ccsn/CCSN_v2/Sc/Sc-N016.jpg  
  inflating: /content/ccsn/__MACOSX/CCSN_v2/Sc/._Sc-N016.jpg  
  inflating: /content/ccsn/CCSN_v2/Sc/Sc-N200.jpg  
  inflating: /content/ccsn/__MACOSX/CCSN_v2/Sc/._Sc-N200.jpg  
  inflating: /content/ccsn/CCSN_v2/Sc/Sc-N214.jpg  
  inflating: /content/ccsn/__MACOSX/CCSN_v2/Sc/._Sc-N214.jpg  
  inflating: /content/ccsn/CCSN_v2/Sc/Sc-N228.jpg  
  inflating: /content/ccsn/__MACOSX/CCSN_v2/Sc/._Sc-N228.jpg  
  inflating: /content/ccsn/CCSN_v2/Sc/Sc-N298.jpg  
  inflating: /content/ccsn/__MACOSX/CCSN_v2/Sc/._Sc-N298.jpg  
  inflating: /content/ccsn/CCSN_v2/Sc/Sc-N273.jpg  
  inflating: /content/ccsn/__MACOSX/CCSN_v2/Sc/._Sc-N273.jpg  
  inflating: /content/ccsn/CCSN_v2/Sc/Sc-N267.jpg  
  inflating: /content/ccsn/__MACOSX/CCSN_v2/Sc/._Sc-N267.jpg  
  inflating: /content/ccsn/CCSN_v2/Sc/Sc-N071.jpg  
  inflating: /content/ccsn/__MACOSX/CCSN_v2/Sc/._Sc-N071.jpg  
  inflating: /c

In [ ]:
def split_data(dataset_dir: str) -> dict[dict[str, list[str]]]:
  splits = {"train": {}, "test": {}, "val": {}}
  sets = {"train": 0.9, "test": 0.05, "val": 0.05}

  for _cls in classes:
    splits["train"].update({_cls: []})
    splits["test"].update({_cls: []})
    splits["val"].update({_cls: []})

    class_path = os.path.join(dataset_dir, _cls)
    images = [join(class_path, f) for f in listdir(class_path) if isfile(join(class_path, f))]
    sets_nums = {k: round(item * len(images)) for k, item in sets.items()}

    nums_sum = sum(list(sets_nums.values()))

    print(f"Images in class {_cls} are distributed as follows: train={sets_nums['train']}, test={sets_nums['test']}, val={sets_nums['val']}, total={nums_sum}. There are {len(images)} images. Any excess images will de added to the val subset")
    print("-"*80)

    for idx, img in enumerate(images):
      if idx <= sets_nums["train"]:
        splits["train"][_cls].append(img)

      elif idx <= sets_nums["train"] + sets_nums["test"]:
        splits["test"][_cls].append(img)

      else:
        splits["val"][_cls].append(img)

  return splits

In [ ]:
def save_data(save_directory: str, path_to_dataset: str) -> None:
  shutil.rmtree(save_directory, ignore_errors=True)

  splits = split_data(path_to_dataset)

  for split, data in splits.items():
        print(f'Saving {split} set')
        print("-"*80)
        split_path = os.path.join(save_directory, split)
        os.makedirs(split_path)

        for _class, images in data.items():
          class_path = os.path.join(split_path, _class)
          os.makedirs(class_path)

          for i in range(len(images)):
            image_path = os.path.join(class_path, f'{i}.png')
            image = Image.open(images[i])
            image.save(image_path)

In [ ]:
def parse_data_with_oversampling(save_directory: str, path_to_dataset: str, oversampling_threshold: float = 0.5) -> None:
  shutil.rmtree(save_directory, ignore_errors=True)

  splits = split_data(path_to_dataset)

  oversampled_images = 0

  assert oversampling_threshold <= 1, "Oversampling threshold has to be in <0,1> range"
  transformer = A.ReplayCompose([
      A.HorizontalFlip(always_apply=False, p=oversampling_threshold),
      A.RandomBrightnessContrast(always_apply=False, p=oversampling_threshold / 3,
                                 brightness_limit=(-0.3, 0.3),
                                 contrast_limit=(-0.3, 0.3),
                                 brightness_by_max=False), #might not be ideal for photos of the sky
  ])

  for split, data in splits.items():
        print(f'Saving {split} set')
        print("-"*80)
        split_path = os.path.join(save_directory, split)
        os.makedirs(split_path)

        for _class, images in data.items():
          class_path = os.path.join(split_path, _class)
          os.makedirs(class_path)

          for i in range(len(images)):
            image_path = os.path.join(class_path, f'{i}.png')
            image = Image.open(images[i])
            image.save(image_path)

            if split == "train":
              oversampled_path = os.path.join(class_path, f'{i}_os.png')
              transformed = transformer(image=np.array(image))

              if transformed["replay"]["transforms"][0]["applied"] or transformed["replay"]["transforms"][0]["applied"]:
                transformed_image = Image.fromarray(transformed['image'])
                transformed_image.save(oversampled_path)
                oversampled_images += 1

  print(f"Saving completed. Oversampled {oversampled_images} images")

In [ ]:
parsed_data_path = "/content/prepared_dataset"

In [ ]:
save_data(parsed_data_path, "/content/no_night_ccsn")

In [ ]:
save_data(parsed_data_path, "/content/ccsn/CCSN_v2")

Images in class Ci are distributed as follows: train=125, test=7, val=7, total=139. There are 139 images. Any excess images will de added to the val subset
--------------------------------------------------------------------------------
Images in class Cs are distributed as follows: train=258, test=14, val=14, total=286. There are 287 images. Any excess images will de added to the val subset
--------------------------------------------------------------------------------
Images in class Cc are distributed as follows: train=241, test=13, val=13, total=267. There are 268 images. Any excess images will de added to the val subset
--------------------------------------------------------------------------------
Images in class Ac are distributed as follows: train=199, test=11, val=11, total=221. There are 221 images. Any excess images will de added to the val subset
--------------------------------------------------------------------------------
Images in class As are distributed as follows:

In [ ]:
parse_data_with_oversampling(parsed_data_path, "/content/ccsn/CCSN_v2", oversampling_threshold=0.8)

Images in class Ci are distributed as follows: train=125, test=7, val=7, total=139. There are 139 images. Any excess images will de added to the val subset
--------------------------------------------------------------------------------
Images in class Cs are distributed as follows: train=258, test=14, val=14, total=286. There are 287 images. Any excess images will de added to the val subset
--------------------------------------------------------------------------------
Images in class Cc are distributed as follows: train=241, test=13, val=13, total=267. There are 268 images. Any excess images will de added to the val subset
--------------------------------------------------------------------------------
Images in class Ac are distributed as follows: train=199, test=11, val=11, total=221. There are 221 images. Any excess images will de added to the val subset
--------------------------------------------------------------------------------
Images in class As are distributed as follows:

In [ ]:
!zip -r /content/oversampled_dataset.zip /content/prepared_dataset

  adding: content/prepared_dataset/ (stored 0%)
  adding: content/prepared_dataset/val/ (stored 0%)
  adding: content/prepared_dataset/val/Ns/ (stored 0%)
  adding: content/prepared_dataset/val/Ns/2.png (deflated 0%)
  adding: content/prepared_dataset/val/Ns/11.png (deflated 0%)
  adding: content/prepared_dataset/val/Ns/9.png (deflated 0%)
  adding: content/prepared_dataset/val/Ns/8.png (deflated 0%)
  adding: content/prepared_dataset/val/Ns/5.png (deflated 0%)
  adding: content/prepared_dataset/val/Ns/0.png (deflated 0%)
  adding: content/prepared_dataset/val/Ns/4.png (deflated 0%)
  adding: content/prepared_dataset/val/Ns/3.png (deflated 0%)
  adding: content/prepared_dataset/val/Ns/7.png (deflated 0%)
  adding: content/prepared_dataset/val/Ns/6.png (deflated 0%)
  adding: content/prepared_dataset/val/Ns/1.png (deflated 0%)
  adding: content/prepared_dataset/val/Ns/10.png (deflated 0%)
  adding: content/prepared_dataset/val/Cu/ (stored 0%)
  adding: content/prepared_dataset/val/Cu/2.

# Model loading & fine-tuning

In [ ]:
model = YOLO('yolov8s-cls.pt')

100%|██████████| 12.2M/12.2M [00:00<00:00, 50.3MB/s]


In [ ]:
model.train(data=parsed_data_path, epochs=10, patience=10, imgsz=400, save=True, save_period=5, verbose=True, seed=42, val=True)

Ultralytics YOLOv8.2.16 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=/content/prepared_dataset, epochs=10, time=None, patience=10, batch=16, imgsz=400, save=True, save_period=5, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

100%|██████████| 6.23M/6.23M [00:00<00:00, 147MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[400] must be multiple of max stride 32, updating to [416]


train: Scanning /content/prepared_dataset/train... 2300 images, 0 corrupt: 100%|██████████| 2300/2300 [00:00<00:00, 3287.06it/s]


train: New cache created: /content/prepared_dataset/train.cache


...

                   all      0.547      0.897



10 epochs completed in 0.140 hours.
Optimizer stripped from runs/classify/train8/weights/last.pt, 10.3MB
Optimizer stripped from runs/classify/train8/weights/best.pt, 10.3MB

Validating runs/classify/train8/weights/best.pt...
Ultralytics YOLOv8.2.16 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-cls summary (fused): 73 layers, 5089291 parameters, 0 gradients, 12.5 GFLOPs
train: /content/prepared_dataset/train... found 2300 images in 11 classes ✅ 
val: /content/prepared_dataset/val... found 117 images in 11 classes ✅ 
test: /content/prepared_dataset/test... found 126 images in 11 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:00<00:00, 11.26it/s]


                   all       0.59      0.906
Speed: 0.6ms preprocess, 1.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train8
Results saved to runs/classify/train8


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ff653c29000>
curves: []
curves_results: []
fitness: 0.7478632628917694
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.5897436141967773, 'metrics/accuracy_top5': 0.9059829115867615, 'fitness': 0.7478632628917694}
save_dir: PosixPath('runs/classify/train8')
speed: {'preprocess': 0.6365388886541383, 'inference': 1.3638011410705044, 'loss': 0.0007295200967381143, 'postprocess': 0.0005338945959368323}
task: 'classify'
top1: 0.5897436141967773
top5: 0.9059829115867615

In [ ]:
print(model.val())

Ultralytics YOLOv8.2.16 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-cls summary (fused): 73 layers, 5089291 parameters, 0 gradients, 12.5 GFLOPs
train: /content/prepared_dataset/train... found 2300 images in 11 classes ✅ 
val: /content/prepared_dataset/val... found 117 images in 11 classes ✅ 
test: /content/prepared_dataset/test... found 126 images in 11 classes ✅ 


val: Scanning /content/prepared_dataset/val... 117 images, 0 corrupt: 100%|██████████| 117/117 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:03<00:00,  2.05it/s]


                   all       0.59      0.906
Speed: 0.3ms preprocess, 5.8ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/classify/train82


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ff5f3a1a4a0>
curves: []
curves_results: []
fitness: 0.7478632628917694
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.5897436141967773, 'metrics/accuracy_top5': 0.9059829115867615, 'fitness': 0.7478632628917694}
save_dir: PosixPath('runs/classify/train82')
speed: {'preprocess': 0.32109684414333767, 'inference': 5.842856871776092, 'loss': 0.0011513375828408787, 'postprocess': 0.10473096472585303}
task: 'classify'
top1: 0.5897436141967773
top5: 0.9059829115867615


Now let's try more epochs

In [ ]:
model2 = YOLO('yolov8s-cls.pt')

In [ ]:
model2.train(data=parsed_data_path, epochs=15, patience=10, imgsz=400, save=True, save_period=5, verbose=True, seed=42, val=True)

Ultralytics YOLOv8.2.16 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=/content/prepared_dataset, epochs=15, time=None, patience=10, batch=16, imgsz=400, save=True, save_period=5, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

train: Scanning /content/prepared_dataset/train... 2300 images, 0 corrupt: 100%|██████████| 2300/2300 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/prepared_dataset/val... 117 images, 0 corrupt: 100%|██████████| 117/117 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 416 train, 416 val
Using 2 dataloader workers
Logging results to runs/classify/train9
Starting training for 15 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:00<00:00, 14.64it/s]

                   all      0.427      0.897


...

                   all       0.53      0.932



15 epochs completed in 0.227 hours.
Optimizer stripped from runs/classify/train9/weights/last.pt, 10.3MB
Optimizer stripped from runs/classify/train9/weights/best.pt, 10.3MB

Validating runs/classify/train9/weights/best.pt...
Ultralytics YOLOv8.2.16 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-cls summary (fused): 73 layers, 5089291 parameters, 0 gradients, 12.5 GFLOPs
train: /content/prepared_dataset/train... found 2300 images in 11 classes ✅ 
val: /content/prepared_dataset/val... found 117 images in 11 classes ✅ 
test: /content/prepared_dataset/test... found 126 images in 11 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:00<00:00, 13.38it/s]


                   all      0.547      0.923
Speed: 0.4ms preprocess, 1.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train9
Results saved to runs/classify/train9


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ff5f395f3a0>
curves: []
curves_results: []
fitness: 0.7350427508354187
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.5470085740089417, 'metrics/accuracy_top5': 0.9230769276618958, 'fitness': 0.7350427508354187}
save_dir: PosixPath('runs/classify/train9')
speed: {'preprocess': 0.4063182406955295, 'inference': 1.3541237920777411, 'loss': 0.0012043194893078925, 'postprocess': 0.0008925413474058494}
task: 'classify'
top1: 0.5470085740089417
top5: 0.9230769276618958

In [ ]:
print(model2.val())

Ultralytics YOLOv8.2.16 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-cls summary (fused): 73 layers, 5089291 parameters, 0 gradients, 12.5 GFLOPs
train: /content/prepared_dataset/train... found 2300 images in 11 classes ✅ 
val: /content/prepared_dataset/val... found 117 images in 11 classes ✅ 
test: /content/prepared_dataset/test... found 126 images in 11 classes ✅ 


val: Scanning /content/prepared_dataset/val... 117 images, 0 corrupt: 100%|██████████| 117/117 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:03<00:00,  2.10it/s]


                   all      0.547      0.923
Speed: 0.6ms preprocess, 4.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train92
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ff5f3b56aa0>
curves: []
curves_results: []
fitness: 0.7350427508354187
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.5470085740089417, 'metrics/accuracy_top5': 0.9230769276618958, 'fitness': 0.7350427508354187}
save_dir: PosixPath('runs/classify/train92')
speed: {'preprocess': 0.602145480294513, 'inference': 4.1184078933846235, 'loss': 0.0010861290825737847, 'postprocess': 0.001163564176640959}
task: 'classify'
top1: 0.5470085740089417
top5: 0.9230769276618958


The results are still not great, so let's try with the bigger model

In [ ]:
model_md = YOLO('yolov8m-cls.pt')

100%|██████████| 32.7M/32.7M [00:00<00:00, 312MB/s]


In [ ]:
model_md.train(data=parsed_data_path, epochs=15, patience=10, imgsz=400, save=True, save_period=5, verbose=True, seed=42, val=True)

Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=/content/prepared_dataset, epochs=15, time=None, patience=10, batch=16, imgsz=400, save=True, save_period=5, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

100%|██████████| 6.23M/6.23M [00:00<00:00, 113MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[400] must be multiple of max stride 32, updating to [416]


train: Scanning /content/prepared_dataset/train... 2300 images, 0 corrupt: 100%|██████████| 2300/2300 [00:00<00:00, 3268.50it/s]


train: New cache created: /content/prepared_dataset/train.cache


...

                   all       0.59      0.974



15 epochs completed in 0.220 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 31.7MB
Optimizer stripped from runs/classify/train/weights/best.pt, 31.7MB

Validating runs/classify/train/weights/best.pt...
Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8m-cls summary (fused): 103 layers, 15776747 parameters, 0 gradients, 41.6 GFLOPs
train: /content/prepared_dataset/train... found 2300 images in 11 classes ✅ 
val: /content/prepared_dataset/val... found 117 images in 11 classes ✅ 
test: /content/prepared_dataset/test... found 126 images in 11 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:00<00:00,  7.10it/s]


                   all      0.615      0.974
Speed: 0.3ms preprocess, 3.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Results saved to runs/classify/train


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7cf244fcfd00>
curves: []
curves_results: []
fitness: 0.7948718070983887
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.6153846383094788, 'metrics/accuracy_top5': 0.9743589758872986, 'fitness': 0.7948718070983887}
save_dir: PosixPath('runs/classify/train')
speed: {'preprocess': 0.29269854227701825, 'inference': 3.7373131156986594, 'loss': 0.0008538238003722624, 'postprocess': 0.00046868609566973825}
task: 'classify'
top1: 0.6153846383094788
top5: 0.9743589758872986

In [ ]:
model_lg = YOLO('yolov8l-cls.pt')

100%|██████████| 71.7M/71.7M [00:00<00:00, 280MB/s]


In [ ]:
model_lg.train(data=parsed_data_path, epochs=15, patience=10, imgsz=400, save=True, save_period=5, verbose=True, seed=42, val=True)

Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8l-cls.pt, data=/content/prepared_dataset, epochs=15, time=None, patience=10, batch=16, imgsz=400, save=True, save_period=5, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

train: Scanning /content/prepared_dataset/train... 2300 images, 0 corrupt: 100%|██████████| 2300/2300 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/prepared_dataset/val... 117 images, 0 corrupt: 100%|██████████| 117/117 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 50 weight(decay=0.0), 51 weight(decay=0.0005), 51 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 416 train, 416 val
Using 2 dataloader workers
Logging results to runs/classify/train2
Starting training for 15 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:01<00:00,  2.81it/s]

                   all      0.462      0.897


...

                   all      0.564       0.94



15 epochs completed in 0.263 hours.
Optimizer stripped from runs/classify/train2/weights/last.pt, 72.6MB
Optimizer stripped from runs/classify/train2/weights/best.pt, 72.6MB

Validating runs/classify/train2/weights/best.pt...
Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l-cls summary (fused): 133 layers, 36198667 parameters, 0 gradients, 98.7 GFLOPs
train: /content/prepared_dataset/train... found 2300 images in 11 classes ✅ 
val: /content/prepared_dataset/val... found 117 images in 11 classes ✅ 
test: /content/prepared_dataset/test... found 126 images in 11 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:00<00:00,  5.55it/s]


                   all      0.615      0.983
Speed: 0.3ms preprocess, 4.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2
Results saved to runs/classify/train2


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7cf1ef2a6470>
curves: []
curves_results: []
fitness: 0.7991453111171722
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.6153846383094788, 'metrics/accuracy_top5': 0.9829059839248657, 'fitness': 0.7991453111171722}
save_dir: PosixPath('runs/classify/train2')
speed: {'preprocess': 0.2812422238863431, 'inference': 4.881956638433994, 'loss': 0.0009801652696397568, 'postprocess': 0.0009108812381059696}
task: 'classify'
top1: 0.6153846383094788
top5: 0.9829059839248657

# Saving model

In [ ]:
export = model.export()

Ultralytics YOLOv8.2.16 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'runs/classify/train8/weights/best.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 11) (9.8 MB)

TorchScript: starting export with torch 2.2.1+cu121...
TorchScript: export success ✅ 0.9s, saved as 'runs/classify/train8/weights/best.torchscript' (19.6 MB)

Export complete (2.5s)
Results saved to /content/runs/classify/train8/weights
Predict:         yolo predict task=classify model=runs/classify/train8/weights/best.torchscript imgsz=416  
Validate:        yolo val task=classify model=runs/classify/train8/weights/best.torchscript imgsz=416 data=/content/prepared_dataset  
Visualize:       https://netron.app


In [ ]:
export = model_md.export()

Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.00GHz)
YOLOv8m-cls summary (fused): 103 layers, 15776747 parameters, 0 gradients, 41.6 GFLOPs

PyTorch: starting from 'runs/classify/train/weights/best.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 11) (30.2 MB)

TorchScript: starting export with torch 2.2.1+cu121...
TorchScript: export success ✅ 1.9s, saved as 'runs/classify/train/weights/best.torchscript' (60.4 MB)

Export complete (4.2s)
Results saved to /content/runs/classify/train/weights
Predict:         yolo predict task=classify model=runs/classify/train/weights/best.torchscript imgsz=416  
Validate:        yolo val task=classify model=runs/classify/train/weights/best.torchscript imgsz=416 data=/content/prepared_dataset  
Visualize:       https://netron.app


In [ ]:
export = model_lg.export()

Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.00GHz)
YOLOv8l-cls summary (fused): 133 layers, 36198667 parameters, 0 gradients, 98.7 GFLOPs

PyTorch: starting from 'runs/classify/train2/weights/best.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 11) (69.2 MB)

TorchScript: starting export with torch 2.2.1+cu121...
TorchScript: export success ✅ 3.5s, saved as 'runs/classify/train2/weights/best.torchscript' (138.4 MB)

Export complete (6.5s)
Results saved to /content/runs/classify/train2/weights
Predict:         yolo predict task=classify model=runs/classify/train2/weights/best.torchscript imgsz=416  
Validate:        yolo val task=classify model=runs/classify/train2/weights/best.torchscript imgsz=416 data=/content/prepared_dataset  
Visualize:       https://netron.app


Test of model loading

In [1]:
import torch
import torch.jit
from torch.utils.mobile_optimizer import optimize_for_mobile
from ultralytics import YOLO
print(torch.__version__)

test_model = YOLO("best.pt")

torch.quantization.prepare(test_model, inplace=True)
torch.quantization.convert(test_model, inplace=True)

try:
    test_model = torch.jit.script(test_model)
    optimized_script = optimize_for_mobile(test_model)
    torch.jit.save(optimized_script, "best_optimized.pt")
except Exception as e:
    print("An error occurred:", e)

2.3.0+cpu
An error occurred: import statements aren't supported:
  File "C:\Studia\8. sem\SIwSI\.venv\lib\site-packages\ultralytics\engine\model.py", line 741
            (list | None): The class names of the model if available, otherwise None.
        """
        from ultralytics.nn.autobackend import check_class_names
        ~~~~ <--- HERE
    
        if hasattr(self.model, "names"):


C:\Studia\8. sem\SIwSI\.venv\lib\site-packages\torch\ao\quantization\quantize.py:312: UserWarning: None of the submodule got qconfig applied. Make sure you passed correct configuration through `qconfig_dict` or by assigning the `.qconfig` attribute directly on submodules
  warnings.warn("None of the submodule got qconfig applied. Make sure you "


In [14]:
results = test_model("test_cloud_1.jpg")

image 1/1 C:\Studia\8. sem\SIwSI\models\test_cloud_1.jpg: 416x416 Cc 0.34, Cu 0.28, Ac 0.17, Cs 0.07, Ns 0.05, 227.1ms
Speed: 30.2ms preprocess, 227.1ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 416)


In [ ]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 masks: None
 names: {0: 'Ac', 1: 'As', 2: 'Cb', 3: 'Cc', 4: 'Ci', 5: 'Cs', 6: 'Ct', 7: 'Cu', 8: 'Ns', 9: 'Sc', 10: 'St'}
 obb: None
 orig_img: array([[[158,  74,  56],
         [156,  74,  56],
         [156,  74,  56],
         ...,
         [159,  83,  61],
         [158,  82,  60],
         [158,  82,  60]],
 
        [[156,  74,  56],
         [156,  74,  56],
         [155,  73,  55],
         ...,
         [160,  81,  60],
         [160,  81,  60],
         [160,  81,  60]],
 
        [[155,  73,  55],
         [155,  73,  55],
         [155,  73,  55],
         ...,
         [160,  81,  60],
         [160,  81,  60],
         [160,  81,  60]],
 
        ...,
 
        [[193, 126, 101],
         [193, 126, 101],
         [194, 127, 102],
         ...,
         [193, 126, 101],
         [193, 126, 101],
         [193, 126, 101]],
 
        [[193, 126, 101],
         [193, 126, 101],
      

In [16]:
classes_map = results[0].names
# print(classes_map)
print(classes_map[results[0].probs.top1])

Cc
